In [191]:
import os
import pandas as pd
import utils
import numpy as np

In [192]:
# files=os.listdir('대구경북')
# for file in files:
#     data=pd.read_csv(f'대구경북/{file}',encoding='cp949')
#     data['일시']=pd.to_datetime(data['일시'])
#     dataDF=data[(9<=data['일시'].dt.month) & (data['일시'].dt.month<=11)]
#     dataDF=dataDF[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 상대습도(%)']]

    

In [193]:
data=pd.read_csv(f'total.csv')
data['일시']=pd.to_datetime(data['일시'])
dataDF=data[(9<=data['일시'].dt.month) & (data['일시'].dt.month<=11)]
dataDF=dataDF[['일시','최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 상대습도(%)']]
dataDF.reset_index(inplace=True,drop=True)

In [194]:
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154997 entries, 0 to 154996
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   일시          154997 non-null  datetime64[ns]
 1   최저기온(°C)    154976 non-null  float64       
 2   최고기온(°C)    154972 non-null  float64       
 3   일강수량(mm)    50635 non-null   float64       
 4   평균 상대습도(%)  154719 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 5.9 MB


## 결측값 처리

In [195]:
# 강수량 nan값은 0로
dataDF['일강수량(mm)']=dataDF['일강수량(mm)'].fillna(0)

In [196]:
# nan 값 확인
nanDF=dataDF[dataDF['최저기온(°C)'].isna()|dataDF['최고기온(°C)'].isna()|(dataDF['평균 상대습도(%)'].isna())]
nanDF

,일시,최저기온(°C),최고기온(°C),일강수량(mm),평균 상대습도(%)
915,2014-09-06,13.4,24.8,0.0,NaN
1112,2016-09-21,8.5,16.8,0.0,NaN
3203,2019-09-19,15.0,26.0,0.0,NaN
9586,2013-10-02,NaN,25.7,4.0,77.1
11780,2017-10-12,8.8,NaN,0.0,71.0
...,...,...,...,...,...
150538,2015-09-01,19.9,23.3,0.0,NaN
154342,2016-11-13,8.0,18.9,0.0,NaN
154406,2017-10-17,3.4,22.2,0.0,NaN
154408,2017-10-19,8.6,11.0,0.0,NaN


In [197]:
# 수치의 일관성이 없어 각 달의 평균값으로 치환
# 각월 평균
mean9=dataDF[dataDF['일시'].dt.month==9].mean()
mean10=dataDF[dataDF['일시'].dt.month==10].mean()
mean11=dataDF[dataDF['일시'].dt.month==11].mean()

In [198]:
for idx in nanDF.index:
    
    for col in dataDF.columns[1:]:
        if ~(-100<dataDF.loc[idx,col]<100):
            if dataDF.loc[idx,'일시'].month==9:
                dataDF.loc[idx,col]=mean9[col]
            elif dataDF.loc[idx,'일시'].month==10:
                dataDF.loc[idx,col]=mean10[col]
            else:
                dataDF.loc[idx,col]=mean11[col]

In [199]:
dataDF.info()
# 결측값 처리 완

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154997 entries, 0 to 154996
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   일시          154997 non-null  datetime64[ns]
 1   최저기온(°C)    154997 non-null  float64       
 2   최고기온(°C)    154997 non-null  float64       
 3   일강수량(mm)    154997 non-null  float64       
 4   평균 상대습도(%)  154997 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 5.9 MB


- 피쳐 9월1일~11월29일
- 타겟 9월1일~11월30일

In [200]:
# 피쳐 데이터셋 제작
featureDF=dataDF[~((dataDF['일시'].dt.month==11) & (dataDF['일시'].dt.day==30))]
featureDF.info()
featureDF.reset_index(inplace=True,drop=True)


<class 'pandas.core.frame.DataFrame'>
Index: 153292 entries, 0 to 154995
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   일시          153292 non-null  datetime64[ns]
 1   최저기온(°C)    153292 non-null  float64       
 2   최고기온(°C)    153292 non-null  float64       
 3   일강수량(mm)    153292 non-null  float64       
 4   평균 상대습도(%)  153292 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 7.0 MB


In [201]:
# 타겟 데이터셋 제작
targetDF=dataDF[~((dataDF['일시'].dt.month==9) & (dataDF['일시'].dt.day==1))]
targetDF.info()
targetDF.reset_index(inplace=True,drop=True)

<class 'pandas.core.frame.DataFrame'>
Index: 153296 entries, 1 to 154996
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   일시          153296 non-null  datetime64[ns]
 1   최저기온(°C)    153296 non-null  float64       
 2   최고기온(°C)    153296 non-null  float64       
 3   일강수량(mm)    153296 non-null  float64       
 4   평균 상대습도(%)  153296 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 7.0 MB


In [202]:
targetDF.loc[12103]

일시            2022-10-15 00:00:00
최저기온(°C)                     12.6
최고기온(°C)                     24.4
일강수량(mm)                      0.0
평균 상대습도(%)                   68.5
Name: 12103, dtype: object

In [220]:
idx=~(targetDF['일시'].dt.dayofyear-featureDF['일시'].dt.dayofyear==1)
print(featureDF[idx].head(2))
print(targetDF[idx].head(2))

Empty DataFrame
Columns: [일시, 최저기온(°C), 최고기온(°C), 일강수량(mm), 평균 상대습도(%)]
Index: []
Empty DataFrame
Columns: [일시, 최저기온(°C), 최고기온(°C), 일강수량(mm), 평균 상대습도(%)]
Index: []


In [219]:
# featureDF=featureDF.drop(index=121012)
# featureDF.reset_index(inplace=True,drop=True)

targetDF=targetDF.drop(index=147200)
targetDF.reset_index(inplace=True,drop=True)

In [205]:
# featureDF=featureDF.drop(index=12103)
# featureDF.reset_index(inplace=True,drop=True)

targetDF=targetDF.drop(index=[59040,62731,62882])
targetDF.reset_index(inplace=True,drop=True)

In [221]:
# csv 파일로 저장
featureDF.to_csv('feature.csv',index=False)
targetDF.to_csv('target.csv',index=False)